In [ ]:
# ! pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs
# ! pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO

 # please re-start kernel if these packages are installed for the first time

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import datetime
import concurrent
from phantasy import caget,fetch_data

In [ ]:
import objFuncs
from objFuncs import maximize_FC
from objFuncs.util import plot_obj_history
from pyBO import pyBO

In [ ]:
n_init = 32
n_local_search = 8
n_budget = 200

# check objFuncs machineIO, source and beam  

In [ ]:
objFuncs._global_machineIO._test = False
objFuncs._global_machineIO._fetch_data_time_span = 2
objFuncs._global_machineIO._ensure_set_timewait_after_ramp : 0.25
# objFuncs._global_machineIO.view()

In [ ]:
SCS = caget("ACS_DIAG:DEST:ACTIVE_ION_SOURCE")
ion = caget("FE_ISRC"+str(SCS)+":BEAM:ELMT_BOOK")
Q = caget("FE_ISRC"+str(SCS)+":BEAM:Q_BOOK")
A = caget("FE_ISRC"+str(SCS)+":BEAM:A_BOOK")
# AQ = caget("FE_ISRC2:BEAM:MOVRQ_BOOK")
AQ = A/Q
ion = str(A)+ion+str(Q)
print('SCS'+str(SCS), ion, 'A/Q=',AQ)

In [ ]:
now0 = datetime.datetime.now()
now0str = now0.strftime('%Y%m%d_%H%M')
fname = now0str+'['+ion+'][pyBO_TR][U-LEBT]FC814_2ndCup_includeSCS2corr'
fname

# preprare objective

In [ ]:
# define control (=decision) knobs
decision_CSETs = [
                  'FE_SCS2:PSC2_D0731:I_CSET',
                  'FE_SCS2:PSC1_D0731:I_CSET',
                  'FE_SCS2:PSC2_D0755:I_CSET',
                  'FE_SCS2:PSC1_D0755:I_CSET',
                  'FE_LEBT:PSC2_D0773:I_CSET',
                  'FE_LEBT:PSC1_D0773:I_CSET',
                  'FE_LEBT:PSC2_D0790:I_CSET',
                  'FE_LEBT:PSC1_D0790:I_CSET',
                  ]

# check duplicates
assert len(decision_CSETs) == len(np.unique(decision_CSETs))

# build boundary around current set
x0,_ = fetch_data(decision_CSETs,0.1)
bounds = []
frac = 0.4
for i,pv in enumerate(decision_CSETs):
    if 'PSC' in pv:
        bounds.append((x0[i]-frac*AQ, x0[i]+frac*AQ))
    else:
        if x0[i]>0:
            bounds.append((0.9*x0[i], 1.1*x0[i]))
        elif x0[i]<0:
            bounds.append((1.1*x0[i], 0.9*x0[i]))
        else:
            bounds.append((-frac*AQ, frac*AQ))
bounds = np.array(bounds)

decision_min = bounds[:,0]
decision_max = bounds[:,1]
list(zip(decision_min, decision_max))  # bounds

In [ ]:
obj = maximize_FC.maximize_FC814(
    decision_CSETs=decision_CSETs,
    decision_min = decision_min,
    decision_max = decision_max,
    objective_goal   = {'FE_LEBT:FC_D0814:PKAVG_RD': {'more than': 40}},
    objective_weight = {'FE_LEBT:FC_D0814:PKAVG_RD': 1},
    objective_norm   = {'FE_LEBT:FC_D0814:PKAVG_RD': 10},
    objective_fill_none_by_init = True,
)

 # Prepare plot callbacks

In [ ]:
# define what to plot
plot_CSETs = plot_obj_history(
                obj.history['decision_CSETs'],
                title = 'decision_CSETs',
                inline = True,
                )
plot_RDs = plot_obj_history(
                obj.history['objective_RDs'],
                title = 'objective_RDs',
                inline = True,
                )
plot_objs = plot_obj_history(
            obj.history['objectives'],
            title = 'objectives',
            inline = True,
            )
callbacks = [plot_CSETs,plot_RDs,plot_objs]

# evaluate objective and plot
def obj_callbacks(x):
    return obj(x,callbacks=callbacks)

# Run BO

### run BO until budget exhaust

In [ ]:
# run optimizer
bo, X_pending, Y_pending_future = pyBO.runBO(
                                    obj_callbacks,  
                                    bounds=obj.decision_bounds,
                                    n_init=n_init,
                                    x0 = obj.x0,
                                    budget = n_init + 2*n_local_search,
                                    batch_size=1,
#                                     timeout = 2+objFuncs._global_machineIO._fetch_data_time_span,
                                    path="./log/",
                                    tag=fname+'_pyBO_history',
                                    write_log = False)


# exploit model for a few iteration
acquisition_func_args = {'beta':0.01}
X_pending, Y_pending_future= bo.loop( 
                                n_loop = 1,  # number of additional optimization interation
                                func_obj = obj_callbacks,
                                bounds = obj.decision_bounds,
                                acquisition_func_args = acquisition_func_args,
                                X_pending = X_pending, 
                                Y_pending_future = Y_pending_future,
                                batch_size = 1,
                                write_log = False,
                                )

for f in callbacks:
    f.close()

### Trust region optimization

In [ ]:
# local optimization near the best_solution
while(len(bo.y)<n_budget):
    print(len(bo.y))
    x_best,y_best = bo.best_sofar()
    bounds_best = np.array(list(zip(x_best-0.25*AQ, x_best+0.25*AQ)))
    acquisition_func_args = {'beta':9}

    X_pending, Y_pending_future= bo.loop( 
                                    n_loop=n_local_search-1,  # number of additional optimization interation
                                    func_obj = obj_callbacks,
                                    bounds = bounds_best,
                                    acquisition_func_args = acquisition_func_args,
                                    X_pending = X_pending, 
                                    Y_pending_future = Y_pending_future,
                                    batch_size = 1,
                                    write_log = False,
                                    )
    
    acquisition_func_args = {'beta':1}

    X_pending, Y_pending_future= bo.loop( 
                                    n_loop=1,  # number of additional optimization interation
                                    func_obj = obj_callbacks,
                                    bounds = bounds_best,
                                    acquisition_func_args = acquisition_func_args,
                                    X_pending = X_pending, 
                                    Y_pending_future = Y_pending_future,
                                    batch_size = 1,
                                    write_log = False,
                                    )
    
x_best,y_best = bo.best_sofar()
bounds_best = np.array(list(zip(x_best-0.2*AQ, x_best+0.2*AQ)))  
acquisition_func_args = {'beta':0.01}

X_pending, Y_pending_future= bo.loop( 
                                n_loop=1,  # number of additional optimization interation
                                func_obj = obj_callbacks,
                                bounds = bounds_best,
                                acquisition_func_args = acquisition_func_args,
                                X_pending = X_pending, 
                                Y_pending_future = Y_pending_future,
                                batch_size = 1,
                                write_log = False,
                                )
    
for f in callbacks:
    f.close()

In [ ]:
ax = bo.plot_obj_history()
ax.vlines(n_init,*ax.get_ylim(),color='k')

In [ ]:
# set to best solution 
x_best,y_best_old = bo.best_sofar()
y_best_new = obj(x_best)
print(x_best,y_best_old[0],y_best_new)   # check if best solution objective value is consistent

In [ ]:
obj.write_log(fname=os.path.join('./log',fname))

# Visualize Surrogate model

In [ ]:
# fixed_values_for_each_dim = {2:x_best[2],3:x_best[3]}  # fix values to visualize high dim surrogate model
fixed_values_for_each_dim = None                         # do not fix values but project maximum. Can take long time to plot
batch_size = 1

In [ ]:
# # plot acuquisition functions for the first 4 epochs. Can take long time for decision dim >= 4
# for epoch in range(4):   
#     fig,ax = plt.subplots(1,batch_size,figsize=(4*batch_size,3))
#     if batch_size ==1:
#         ax = [ax]
#     for i in range(batch_size):
#         bo.plot_acquisition_2D_projection(project_maximum=True,
#                                           epoch=epoch,
#                                           i_query=i,
#                                           grid_ponits_each_dim = 16,
#                                           fixed_values_for_each_dim=fixed_values_for_each_dim,
#                                           fig=fig,ax=ax[i],);
#         ax[i].legend()
#         ax[i].set_xlabel(obj.decision_CSETs[0])
#         ax[i].set_ylabel(obj.decision_CSETs[1])
#     fig.tight_layout()

In [ ]:
# # plot acuquisition functions for the last 4 epochs. Can take long time for decision dim >= 4
# for epoch in range(-6,-2):
#     fig,ax = plt.subplots(1,batch_size,figsize=(4*batch_size,3))
#     if batch_size ==1:
#         ax = [ax]
#     for i in range(batch_size):
#         bo.plot_acquisition_2D_projection(project_maximum=True,
#                                           epoch=epoch,
#                                           i_query=i,
#                                           grid_ponits_each_dim = 16,
#                                           fixed_values_for_each_dim=fixed_values_for_each_dim,
#                                           fig=fig,ax=ax[i],);
#         ax[i].legend()
#         ax[i].set_xlabel(obj.decision_CSETs[0])
#         ax[i].set_ylabel(obj.decision_CSETs[1])
#     fig.tight_layout()

In [ ]:
# # plot surrogate mean model of the last epoch. Can take long time for decision dim >= 4

# t0 = time.monotonic()
# from math import ceil
# nplot = int(0.5*len(obj.decision_CSETs))
# nrow = ceil(0.5*nplot)
# fig,ax = plt.subplots(nrow,2,figsize=(8,3*nrow))
# for i in range(nrow):
#     for j in range(2):
#         n = 2*i+j
#         if nrow>1:
#             ax_ = ax[i,j]
#         else:
#             ax_ = ax[j]
#         if n >= nplot:
#             ax_.set_visible(False)
#             break
#         bo.plot_model_2D_projection(project_maximum=True,
#                                     dim_xaxis = 2*n,
#                                     dim_yaxis = 2*n+1,
#                                     grid_ponits_each_dim = 16,
#                                     fixed_values_for_each_dim=fixed_values_for_each_dim,
#                                             fig=fig,ax=ax_);
#         ax_.set_xlabel(obj.decision_CSETs[2*n  ])
#         ax_.set_ylabel(obj.decision_CSETs[2*n+1])
#         ax_.legend()
# fig.tight_layout()

    
# print('time took: ',time.monotonic()-t0)